# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd #в начале любого кода импортируем необходимые библиотеки и задаем общепринятые для них сокращения - по ним, в дальнейшем, будем обращаться к библиотекам
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv') # прочитаем с помощью pd нужным датасет, указав "путь" к нему

В качестве ознакомления с данными, я решил посмотрел по 10 строк, как с "головы" таблицы, так и с "хвоста". Возможно, ранее уже была совершена какая-то упорядоченность, но эта теория не подтвердилась.

In [3]:
data = pd.read_csv('/datasets/data.csv') # прочитаем с помощью pd нужным датасет, указав "путь" к нему
display(data.head(10))
display(data.tail(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Следующим шагом нужно получить общую информацию о таблице, применив метод info()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


На данном этапе необходимо сверить, соответствуют ли фактические данные тем типам, на которые указал info(). Если же есть расхождения - это повод устранить данную оплошность на стадии предобработки данных.

**Вывод**

**При первичном взгляде на датафрейм в глаза бросается**:
- Отрицательное значение чисел, хотя значение этого стобца не может быть отрицательным,
- Много вариаций уровней образования заемщиков, которые написаны разным регистром. По смыслу, значений намного меньше.
- Также, в столбце со трудовым стажем заемщиков в днях есть некорректные значения, например, у людей 400000 дней рабочего стажа, что невозможно физически.
- Множество синонимов в столбце, где хранятся цели получения займа.
- Есть значения с плавающей точкой там, где целесообразнее оставить целые числа.

Прежде, чем перейти к следующему шагу были просмотрены все столбы методом unique, с целью не захламлять проект я не буду сюда включать эти действия, но опишу найденные проблемы перед их устранением.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
data.isna().sum() #метод isna найдет пропуски, а sum - сложит результат работы isna

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски имеются в двух столбцах. Если "days_employed" в рамках данного исследования не должен исказить результат, поскольку нужно установить зависимость между фактом неплатежеспособности и различных обстоятельств. Трудовой стаж важен для принятия решения, выдавать ли человеку кредит, а это не является предметом исследования. Но, известен возраст заемщика (dob_years), можно попробовать решить обратную задачу. Предположим, что заемщики начали трудовой стаж в 18 лет, разумеется, это не точно, но предположение не должно исказить суть исследования. Однако следует понимать, что это не самый достоверный метод, будь он целью исследования - потребовались бы точные значения.

Ежемесячный доход (total_income), в свою очередь, более интересен, т.к. зависимость неплатежеспособности от дохода - цель исследования. Заполним пропуски медианным значением

Сперва, разберемся с пропусками в days_employed:

In [6]:
data['days_employed'] = data['dob_years']-18 #вычтем 18 из возраста заемщика. Можно было перезаписать значения столбца применив ко всем значения одно математическое выражение, но чтобы не ошибиться, сделаю последовательный расчет
#display(data['days_employed'].unique()) #встретилось одно значение -18. Этому заемщику исполнилось 18, либо он еще не начал карьеру, либо стаж крайне мал. В основном, банки предъявляют требование не менее полугода/6 месяцев на последнем месте работы, округлим до 1. Вывод на экран закомментирую, выведу итоги манипуляций.
data['days_employed'] = data['days_employed'].replace(-18, 1) #методом replace заменим проблемное значение. Получим стаж заемщиков в годах и переведем в дни.
data['days_employed'] = data['days_employed']*365
display(data['days_employed'].sort_values(ascending=False)) #выведем значения стажа работников в днях в порядке убывания

8880     20805
4895     20440
2557     20440
3460     20440
12317    20440
         ...  
149        365
18415      365
19371      365
1149       365
14514      365
Name: days_employed, Length: 21525, dtype: int64

Самый "опытный" заемщик проработал 57 лет - похоже на правду, в отличие, от исходных 400000 рабочих дней и более.

Пропусков в этом столбце больше нет. Однако, если в дальнейшем эта информация потребуется для другого исследования - придется обратиться к "автору" датафрейма, с целью получения корректной информации по поводу пропусков.

Перейдем к устранению пропусков в столбце с доходами. В идеале, стоит обратиться к сотруднику, имеющему отношение к формированию датафрейма. Возможно, есть закономерность пропуска значений. Например: у тех людей, чьи доходы не заполнены заработок равен МРОТ. Узнав МРОТ - можно будет заполнить эти пропуски точным значением, и.т.п. Однако, раз связи с разработчиком нет - придется заполнить пропуски медианным значением. Доходы у большинства заемщиков все же известны и такой способ борьбы с пропусками не должен исказить итоги исследования.

In [7]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type', 'education'])['total_income'].transform('median'))
data['total_income'].isna().sum() #проверим, есть ли пропуски

0

**Вывод**

Возможно, существует технологическая причина, почему в двух столбцах. где встретились пропущенные значения равное количество пропусков. Скорее всего, у одних и тех же людей пропущены как стаж, так и заработок. Это могло произойти в результате переноса части информации в другой датафрейм, например, перенесли заемщников, кто не платил более 3-х месяцев. Если "дело терпит", равно как и есть вероятность проведения другого исследования с этими данными - следует пообщаться с ответственными лицами, которые имели отношение к формированию и выгрузке данных и узнать, как с этим бороться, без потерь "комплексности" данных.

### Замена типа данных

Для того, чтобы все расчеты были корректны и не было вероятности "поймать ошибку", что совершаются манипуляции не с теми данными, проверим, все ли значения соответствуют типу данных. Бывает так, что числа воспринимаются как строка - это будет причиной для ошибки. Применим метод info

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Переведем значения столбца total_income в int. Крайне маловероятно, что в банковском деле нужна точность дохода вплоть до дробной части.
**Примечание:** до работы со значениями стажа работников в трудовых днях там тоже был тип float, но сейчас все значения целые.

In [9]:
data['total_income'] = data['total_income'].astype('int')

In [10]:
data.info() #проверим изменения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Проблема решена.

**Прочие наблюдения:** есть нестадартное значение в столбце, хранящем информацию о поле заемщика (помимо M и F встречается XNA - возможно сбилась кодировка, и.т.п.) и нелогичные значения столбца, содержащем информацию о количестве детей (есть отрицательное число, а у одного заемщика количество детей равно 20. Скорее всего, тут человеческий фактор - случайно набрали 0.

In [11]:
data['gender'] = data['gender'].replace('XNA', 'unknown') #поскольку такая проблема встречается один раз, а для банковского исследования пол заемщика вряд ли играет решающую роль - заменим на unknows, методом replace
display(data['gender'].unique()) # проверим итог

array(['F', 'M', 'unknown'], dtype=object)

In [12]:
data['children'] = data['children'].replace(20, 2) #также применим метод replace
data['children'] = data['children'].abs() #вызовем фукцию, возвращающую модуль числа. В принципе, можно было также заменить через replace
display(data['children'].unique()) # проверим итог

array([1, 0, 3, 2, 4, 5])

**Вывод**

Бывает так, что числовые значения имеют тип данных, как у строк - что может стать причиной "поломки" кода. Поэтому, нужно переводить данные в соответствующий им тип. Также, если дробная часть не исказит итоги работы - лучше оставить целые значения. Причиной возникновения пропусков стали технологические ошибки на прошлых стадиях работы с данными, человеческий фактор тоже мог иметь место.

### Обработка дубликатов

Сперва, проверим наличие дубликатов в других столбцах. При просмотре начала и конца таблицы сразу бросилось в глаза множество вариаций формулировок уровня образования. Вызовем метод value_counts(), который выведет количество "упоминаний" формулировок.

In [13]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Предположение подтвердилось: на каждый уровень образования есть несколько формулировок, связанных с регистром. Целесообразно все привести к "строчному" виду.

In [14]:
data['education'] = data['education'].str.lower() # метод сделает строчным все значения столбца
display(data['education'])

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21525, dtype: object

In [15]:
data['education'].value_counts() #снова вызовем value_counts, чтобы посмотреть, что изменилось. Должно остаться по одной формулировке, на каждый уровень образования

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**В дополнение к проблеме с регистром:** в столбце 'family_status', нет дубликата, как такового, но одна из формулировок семейного положения заемщика начинается с заглавной буквы, а все остальные строчные. Считаю, надо стремиться к единому стилю во всех "областях" данных, поэтому, поправим регистр и здесь.

In [16]:
data['family_status'] = data['family_status'].str.lower()
display(data['family_status'].unique()) #все значения будут строчными

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Затем, удалим явные дубликаты. Для их идентификации применим связку методов duplicated() и sum()

In [17]:
data.duplicated().sum()

54

In [18]:
data.drop_duplicates(keep=False,inplace=True)
data.duplicated().sum() # по итогу должно быть 0

0

**Вывод**

Скорее всего, в ситуции с регистром уровней образования замещиков имел место человеческий фактор. Люди, заполнявшие данные не выдерживали единый стиль оформления данных и писали по своему. Отсюда - столько формулировок одного и того же значения.

Причиной явных дубликатов тоже может быть человеческий фактор или, например, неудачное слияние таблиц, и.т.п.

### Лемматизация

Приступим к обработке синонимов в столбце с целями, на которые заемщик берет кредит. Просмотрев 20 строк датафрейма создалось впечатление, что фактически одно назначение написано в многократных вариациях, но смысл не меняется. Применим лемматизацию.

**Небольшая аннотация:** попробовав применить лемматизацию ко всему столбцу разом получается ошибка. Поэтому, сперва создадим список со значениями, а затем будем лемматизировать уникальные значения в полученном списке.

In [19]:
purpose_list = data['purpose'].unique()
display(purpose_list)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
lemmas = [] #создадим пустой список и наполним его итогами лемматизации, применив метод append

for lemmas_value in purpose_list:
    lemma = m.lemmatize(lemmas_value)
    lemmas.append(lemma)
display(lemmas)

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

**Прослеживаются фактически 4 цели, связанные с:**
- автомобилями,
- недвижимостью,
- образованием,
- свадьбой.

Лемматизация подтвердила предположение, что по сути, в столбце 4 цели для получения займа. Более 21000 значений переписать вручную весьма тяжело. Стоит применить функцию, для замены целей кредитования в едином стиле, с учетом 4-х выявленных целей по итогам лемматизации

In [21]:
def correct_purpose(row): #объявляем функцию с именем "корректная цель"
    
    if 'авто' in row: #здесь и далее указываем, как объект поиска корень слова, т.к. расхождения в окончаниях
        return "автомобиль"
    if "образов" in row:
        return "образование"
    if "свад" in row:
        return "свадьба"
    if "недвижим" or "жил" or "строит" in row: #все перечисленные цели относятся к недвижимости, достаточно любой из них, поэтому используем "or"
        return "недвижимость"
    
data['correct_purpose'] = data['purpose'].apply(correct_purpose) #создадим новый столбец с правильными целями и наполним его результатами работы функции
display(data.sample(10))    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,correct_purpose
5869,1,5840,34,высшее,0,в разводе,3,F,сотрудник,0,144683,строительство недвижимости,недвижимость
16149,1,4015,29,среднее,1,женат / замужем,0,F,сотрудник,0,81764,на покупку своего автомобиля,автомобиль
66,0,3650,28,среднее,1,женат / замужем,0,M,сотрудник,0,145214,заняться образованием,образование
1599,0,13505,55,высшее,0,женат / замужем,0,F,пенсионер,0,206315,покупка жилья,недвижимость
1022,2,6570,36,среднее,1,гражданский брак,1,F,сотрудник,0,91245,сыграть свадьбу,свадьба
14807,1,1825,23,среднее,1,не женат / не замужем,4,F,компаньон,0,107909,покупка жилья для семьи,недвижимость
6932,0,15695,61,среднее,1,женат / замужем,0,F,пенсионер,0,74546,покупка жилой недвижимости,недвижимость
3904,0,2190,24,среднее,1,гражданский брак,1,F,сотрудник,0,181179,сыграть свадьбу,свадьба
9726,0,10585,47,среднее,1,женат / замужем,0,F,компаньон,0,105822,строительство жилой недвижимости,недвижимость
3903,0,6570,36,среднее,1,женат / замужем,0,M,сотрудник,0,136818,на покупку автомобиля,автомобиль


**Вывод**

Причина возникновения такого количества дубликатов в целях получения займа клиентом идентична проблеме с уровнем образования - человеческий фактор. Очевидно, что при "занесении" информации в столбец purpose персоналом не применялись никакие единый методики оформления - каждый писал "на свой вкус", что повлекло столько синонимов, а также, орфографические ошибки.

### Категоризация данных

Для проверки гипотез необходимо также произвести категоризацию значений дохода у замещиков - в столбце огромное количество уникальных значений. Для этого, целесообразно создать функцию, разбив доходы на 3 равные категории. Затем, итоги работы функции передать во вновь созданный столбец.

In [22]:
income_str = pd.qcut(data['total_income'], 3, ['Низкий доход', 'Средний доход', 'Высокий доход']) #qcut разделит данные на заданное количество компонентов. В моем случае их 3
income_str.value_counts() #проверим значения.

Низкий доход     7140
Высокий доход    7140
Средний доход    7139
Name: total_income, dtype: int64

3 полученные категории назовем как низкий, средний и высокие доходы.

In [23]:
def structured_income (income):  #назовем функцию "структурированный доход"
    if income < 119966: #работать будем с ежемесячным заработком заемщика
        return('Низкий заработок')
    if 119966 <= income <= 173504:
        return('Средний заработок')
    if 173504 <= income <= 2265604:
        return('Высокий заработок')
    
data['structured_income'] = data['total_income'].apply(structured_income)
display(data['structured_income']) #проверим

0        Высокий заработок
1         Низкий заработок
2        Средний заработок
3        Высокий заработок
4        Средний заработок
               ...        
21520    Высокий заработок
21521    Средний заработок
21522     Низкий заработок
21523    Высокий заработок
21524     Низкий заработок
Name: structured_income, Length: 21419, dtype: object

In [24]:
data['structured_income'].value_counts()

Низкий заработок     7140
Высокий заработок    7140
Средний заработок    7139
Name: structured_income, dtype: int64

**Вывод**

Вероятнее всего, в банковском деле, необходимо учитывать индивидуальные значения дохода для каждого клиента. Однако, для установления зависимости количества нестабильных заемщиков и уровня их дохода, целесообразнее выявить категорию.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Поскольку, для проверики гипотез не требуется работа со всеми данными, целесообразно "разбить" датафрейм, создав таблицу только с теми данными, которые необходимы.

In [25]:
arrears_children = data[['children', 'debt']] #создаем новый df с данными неплательщиков и включаем данные по "детям" и факту задолженности

borrower_with_children = arrears_children.loc[arrears_children['children']>0] # в эту переменную сохраним только тех заемщиков, у которых есть дети (кол-во более 0)
defaulter_with_children = borrower_with_children.loc[borrower_with_children['debt'] == 1] # "выявим" клиентов с детьми, у которых есть факт задолженности
ratio_borrower_with_children = len(defaulter_with_children) / len(borrower_with_children) # Вычислим соотношение между должниками и добросовестными плательщиками без детей. С помощью len определим количество "совпадений" по заданным условиям

#аналогичный подсчет для клиентов без детей (кол-во = 0)

borrower_without_children = arrears_children.loc[arrears_children['children']==0]
defaulter_without_children = borrower_without_children.loc[borrower_without_children['debt'] == 1]
ratio_borrower_without_children = len(defaulter_without_children) / len(borrower_without_children)

if ratio_borrower_with_children > ratio_borrower_without_children:
    print('Процент недобросовестных заемщиков с детьми составляет - {:.1%}, это больше, чем количество невыполнения платежных обязательств среди заемщиков без детей.'.format(ratio_borrower_with_children))
else:
    print('Процент недобросовестных заемщиков без детей составляет - {:.1%}, это больше, чем количество невыполнения платежных обязательств среди заемщиков с детьми'.format(ratio_borrower_without_children))

Процент недобросовестных заемщиков с детьми составляет - 9.2%, это больше, чем количество невыполнения платежных обязательств среди заемщиков без детей.


**Факт несвоевременной выплаты чаще встречается у плательщиков с детьми:** 9% против 7,5%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
defaulter_family_status = data[['family_status', 'debt']] #будем использовать "выжимку" только со столбцами с семейным положением и фактом задолженности
defaulter_family_status_grouping = defaulter_family_status.groupby('family_status').count() # сгруппируем "новый" датафрейм по столбцу с семейным положением + найдем количество, когда "встречается" заданное условие
defaulter_family_status_grouping['quantity_debtors'] = defaulter_family_status.loc[defaulter_family_status['debt'] == 1].groupby('family_status').sum() #посчитаем количество "просрочек", и сгруппируем по семейному положению
defaulter_family_status_grouping['percent (%)'] = (defaulter_family_status_grouping['quantity_debtors'] / defaulter_family_status_grouping['debt']) * 100  #поделим количество должников на количество фактов не выплат и умножим на 100 - это будет процентное соотношение
display(defaulter_family_status_grouping.sort_values('percent (%)', ascending=False)) #отсортируем столбец с количеством задолженности в обратном порядке - на первом месте будут самые "злостные" неплательщики, в зависимости от семейного положения

,debt,quantity_debtors,percent (%)
family_status,,,
не женат / не замужем,2807,274,9.761311
гражданский брак,4149,388,9.351651
женат / замужем,12310,931,7.562957
в разводе,1195,85,7.112971
вдовец / вдова,958,63,6.576200


**Вывод**

**Наиболее часто люди нарушают свои кредитные обязательства, не находясь в браке**, на втором месте те, что пребывает в "неофициальном браке", их количество немногим меньше. Люди, состоящие в браке нарушают платежеспособность гораздо реже.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для проверки зависимости цели кредования и регулярности выплат возпользуемся вновь созданным столбцом, в котором уровень заработка разбит на категории - structured_income

In [27]:
defaulter_income = data[['structured_income', 'debt']] #также используем часть данных. В этом случае - категорированные уровни дохода и наличие задолженности
defaulter_income_grouping = defaulter_income.groupby('structured_income').count()
defaulter_income_grouping['quantity_debtors'] = defaulter_income.loc[defaulter_income['debt'] == 1].groupby('structured_income').sum()
defaulter_income_grouping['percent (%)'] = (defaulter_income_grouping['quantity_debtors'] / defaulter_income_grouping['debt']) * 100


display(defaulter_income_grouping.sort_values('percent (%)', ascending = False))

,debt,quantity_debtors,percent (%)
structured_income,,,
Средний заработок,7139,633,8.866788
Низкий заработок,7140,579,8.109244
Высокий заработок,7140,529,7.408964


**Вывод**

Данные говорят о том, что **чаще всего нарушают платежную дисциплину клиенты, доходы которых можно отнести к среднему заработку (минимальный доход ~120000 рублей)**. Меньше всего нарушений - у лиц, с высоким заработком.

- Как разные цели кредита влияют на его возврат в срок?

Для решения поставленного вопроса целесообразно применить метод "pivot_table" - сводная таблица.
Создадим таблицу, где:
- сгруппируем данные по лемматизированным целям кредита
- в качестве столбца используем значения debt,т.е. есть ли задолженность у заемщика
- к значениям применим функцию sum или суммирование

In [28]:
purpose_credit = data[['correct_purpose', 'debt']]
purpose_credit_grouping = purpose_credit.groupby('correct_purpose').count()
purpose_credit_grouping['quantity_debtors'] = purpose_credit.loc[purpose_credit['debt'] == 1].groupby('correct_purpose').sum()
purpose_credit_grouping['percent (%)'] = (purpose_credit_grouping['quantity_debtors'] / purpose_credit_grouping['debt']) * 100

display(purpose_credit_grouping.sort_values('percent (%)', ascending=False))

,debt,quantity_debtors,percent (%)
correct_purpose,,,
автомобиль,4301,403,9.369914
образование,4006,370,9.236146
свадьба,2322,186,8.010336
недвижимость,10790,782,7.247451


**Вывод**

Чаще всего люди не выполняют платежные обязательства, если кредит был оформлен на **решение вопросов с личными автомобилями.** Также. "вторым" по невозвратности становится кредит на образование.

## Шаг 4. Общий вывод

В исходных данных встретилось много пропусков и недостоверных значений, которые видны, даже не находясь в банковской сфере деятельности. Причинами этого факта, можно считать как технологические ошибки, так и человеческий фактор, в равной приоритетности.

**В процессе выполнения проектов была произведена работа:**
- по "приведению" данных в порядок
- устранение дубликатов, допущенных человеком (разный регистр и множество синонимов)
- проведена лемматизация, с помощью функции столбец с целями приведен к единому стилю
- проверены гипотезы и установлены зависимости между фактом просроченной задолженности и различным обстоятельствам плательщика.

***Спасибо за внимание!***